In [216]:
def parse_text(text: str) -> str:
    return (
        text.replace("**", "{b}")
        .replace("*", "{i}")
        .replace('"', '\\"')
        .replace("%", "\\%")
    )

def close_tags(text: str) -> str:
    _open = False
    open_chars = set()
    output = ""
    for char in text:
        if char == "{":
            _open = True
        elif char == "}":
            _open = False
        elif _open:
            if char not in open_chars:
                open_chars.add(char)
                output += "{" + char + "}"
            else:
                open_chars.remove(char)
                output += "{/" + char + "}"
        else:
            output += char
    return output

skip_keywords = ["!", ".INT", ".SELECT", ".MINI GAME"]

characters = {
    "DAISY-BOT": ("daisy", "left"),
    "BATTA S. DEVILA": ("batta", "right"),
    "BATTA S. D DEVILA": ("batta", "right"),
    "MOON DOG CHEF": ("moondog", "center"),
    "TULONG": ("tulong", "center"),
    "HELL O. WORLD": ("helloworld", "center"),
    "CANDLESTEIN": ("candlestein", "center"),
    "ARCHCLERIC": ("archcleric", "center"),
    "GROUP OF MINIONS": ("minions", "center"),
}

side_char_pos_priority = ["right", "center", "right1", "right2"]

rooms = {
    "BEDROOM, DAWN": "bedroom dawn",
    "BEDROOM, EARLY EVENING": "bedroom early evening",
    "HALLWAY, MIDNIGHT": "hallway midnight",
    "HALLWAY, DAWN": "hallway dawn",
    "CHAPEL, MIDNIGHT": "chapel midnight",
    "CHAPEL, DAWN": "chapel dawn",
    "KITCHEN, MIDNIGHT": "hallway midnight",  # no kitchen scene
    "KITCHEN, DAWN": "hallway dawn",  # no kitchen scene
}

In [217]:
with open("Toxic Yuri Game Jam 2025_ Script.fountain", "r") as file:
    text = file.read()

In [220]:
output: str = f"label start:\n"
character = None
current_string = ""
characters_present = set()
character_expressions = {}
character_positions = {}
all_scenes = set()
all_character_sprites = set()
in_menu = False
TAB = "    "
ctab = TAB
level = 1


def get_open_spot():
    for option in side_char_pos_priority:
        if option not in character_positions.values():
            return option
    return "center"


for line in text.split("\n"):
    line = line.strip()

    if not line:
        if character and current_string:
            output += f"{ctab}{character} \"{close_tags(current_string.strip())}\"\n{ctab}\n"
        elif current_string:
            output += f"{ctab}\"{close_tags(current_string.strip())}\"\n{ctab}\n"
        character = None
        current_string = ""
        continue


    if character:
        current_string += parse_text(line) + " "
    elif line == "NARRATOR":
        continue  # Skip narrator lines, since they are processed normally with no character

    else:
        character_found = False
        for char_text, (name, pos) in characters.items():
            if line.startswith(char_text):
                character = name

                expression = ""
                if "(O.S.)" in line:
                    if character in characters_present:
                        output += f"{ctab}hide {name} with fastdissolve\n"
                        characters_present.remove(character)
                        character_positions.pop(character, None)
                        character_expressions.pop(character, None)
                elif "(" in line:
                    current_expression = character_expressions.get(character)
                    expression = " " + (
                        line[line.find("(") + 1 : line.find(")")].replace("-", "").replace(",", "").lower()
                    )
                    if not current_expression or current_expression != expression:
                        character_expressions[character] = expression

                    else:
                        expression = ""
                all_character_sprites.add(name + expression)

                if character not in characters_present:
                    if pos == "center":
                        pos = get_open_spot()
                    if character == "batta":
                        for other_char, other_pos in character_positions.items():
                            if other_pos == pos and other_char != "batta":
                                other_pos = get_open_spot()
                                output += f"{ctab}show {other_char} at {other_pos} with move\n"
                                character_positions[other_char] = other_pos
                    output += f"{ctab}show {name}{expression} at {pos} with fastdissolve\n"
                    if [c for c, p in character_positions.items() if p == pos and c != character]:
                        print("HEY! Multiple characters at the same position:", pos, character, [c for c, p in character_positions.items() if p == pos and c != character])
                    characters_present.add(character)
                    character_positions[character] = pos
                elif expression:
                    output += f"{ctab}show {name}{expression}\n"
                break


        if not character:
            if any(line.startswith(keyword) for keyword in skip_keywords):
                if line.startswith(".INT"):
                    in_menu = False
                    for room_text, room_name in rooms.items():
                        if room_text in line:
                            output += f"{ctab}scene {room_name} with fade\n"
                            all_scenes.add(room_name)
                            characters_present.clear()
                            character_expressions.clear()
                            character_positions.clear()
                            break
                elif line.startswith("!"):
                    line = line[1:]
                    output += f"{ctab}# {line}\n"
                    if "leaves" in line:
                        character_leaving = line.split(" ", 1)[0].lower().strip()
                        for character in characters_present:
                            if character.startswith(character_leaving):
                                output += f"{ctab}hide {character} with fastdissolve\n"
                                characters_present.remove(character)
                                character_positions.pop(character, None)
                                character_expressions.pop(character, None)
                                break
                elif line.startswith(".MINI GAME"):
                    if "RICE" in line:
                        output += f"{ctab}call screen rice_counting_game\n"
                    elif "GARLIC" in line:
                        output += f"{ctab}call screen pizza_degarlicking_game\n"
                    elif "RELICS" in line:
                        output += f"{ctab}call screen destroy_relics_game\n"
                elif line.startswith(".SELECT"):
                    if not in_menu:
                        in_menu = True
                        level += 2
                        ctab = TAB * level
                        last_newline = output.rfind('\n', 0, len(output) - 1)
                        if last_newline != -1:
                            output = output[:last_newline] + f"menu:\n{TAB}" + output[last_newline + 1:]
                    option_text = line.replace(".SELECT: ", "")
                    if_text = ""
                    if "DANGEROUS ITEMS" in option_text:
                        if_text = " if has_all_dangerous_items()"
                    elif "UPGRADE ITEMS" in option_text:
                        if_text = " if has_all_upgrade_items()"
                    output += (
                        f'{TAB * (level - 1)}"{parse_text(option_text)}"{if_text}:\n'
                    )
                continue
            output += f"{ctab}\"{parse_text(line)}\"\n"

            continue
output += f"{TAB}return\n"
print(output)
# print("\n".join(sorted([c + ".png" for c in all_character_sprites])))

label start:
    scene bedroom early evening with fade
    # OPEN on the bedroom of COUNTESS BATTA S. DEVILA, a Villaness who lives in a comically- spooky castle on the edge of the kingdom. Despite the keep's looming exterior, what we can see of Batta's room is a bit of a hodgepodge of different styles and interests- from the billowing, dramatic Victorian curtains to an evil Lightning McQueen racecar bed in the center of the room.
    # This evening, Batta awakens from her evil slumber and begins preparing for the night ahead with the help of her trusty homemade vampiric thrall, DAISY-BOT.
    show batta relaxed happy at right with fastdissolve
    batta "Oh DAISY-BOT, my precious little iron pill, you would not {i}believe{/i} the dream I just had-"
    
    show daisy neutral at left with fastdissolve
    daisy "My Lady, surely this might not be another one of your {i}fantasy dreams?{/i} Because I feel as your Chief of Staff, it is my duty to remind you that the floors in the West Hal